In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# import CRUD Python Module file and class name
from Module_4_Milestone import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# updated username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True, errors='ignore')

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Define dashboard layout
app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    # dashboard title
    html.Center(html.B(html.H1('CS-340 Dashboard - Brooklynn Seller'))),
    # Logo linked to URL
    html.Center(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
            href="https://www.snhu.edu", 
            # open URL in new tab
            target='_blank')
    ),
    html.Hr(),
        
    # Rescue filter radio buttons
    html.Div([
        html.Label('Filter by Rescue Type:'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset', 'value': 'All'},
                {'label': 'Water Rescue', 'value': 'water_rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain_wilderness_rescue'},
                {'label': 'Disaster or Individual Tracking Rescue', 'value': 'disaster_individual_tracking'},
            ],
            value='All',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        )
    ], style={'padding': '10px 20px'}),
    html.Hr(),
    
    # Interactive data table
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10
    ),    
    html.Br(),
    html.Hr(),
# Side-by-side chart and map
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################   
# Callback to update table data based on rescue type
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
# Filters interactive data table with MongoDB queries
# Displays the breeds of animal based on quantity represented in
# the data table
    if filter_type == 'All':
        df_out = pd.DataFrame.from_records(db.read({}))
    elif filter_type == 'water_rescue':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever", 
                "Newfoundland"]}, "age_upon_outcome_in_weeks": {"$gte": 26, "$lt": 156},
                "sex_upon_outcome": "Intact Female"}
        df_out = pd.DataFrame.from_records(db.read(query))
    elif filter_type == 'mountain_wilderness_rescue':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", 
                "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", 
                "Rottweiler"]}, "age_upon_outcome_in_weeks": {"$gte": 26, "$lt": 156},
                "sex_upon_outcome": "Intact Male"}
        df_out = pd.DataFrame.from_records(db.read(query))
    elif filter_type == 'disaster_individual_tracking':
        query = {"animal_type": "Dog", "breed": {"$in": 
                ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", 
                 "Rottweiler"]}, "age_upon_outcome_in_weeks": {"$gte": 20, "$lt": 300},
                "sex_upon_outcome": "Intact Male"}
        df_out = pd.DataFrame.from_records(db.read(query))
    else:
        df_out = pd.DataFrame.from_records(db.read({"animal_type": filter_type}))
    
    df_out.drop(columns=['_id'], inplace=True, errors='ignore')
    return df_out.to_dict('records')

# Callback to update pie chart based on selected rescue option
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # pie chart to display results of filtered data
    if viewData is None or len(viewData) == 0:
        return html.Div(["No data to display"])

    dff = pd.DataFrame.from_dict(viewData)
    if 'breed' not in dff.columns:
        return html.Div(["'breed' column not found"])
    # title of pie chart
    fig = px.pie(dff, names='breed', title='Preferred Animals')
    return [dcc.Graph(figure=fig)]

#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:15040/
